this script estimates the importance of the funding resources 

In [1]:
library(fixest)
library(tidyr)
library(ggplot2)
library(dplyr)
library(modelsummary)
library(kableExtra)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘kableExtra’


The following object is masked from ‘package:dplyr’:

    group_rows




In [2]:
data = read.csv('regression_data.csv')

In [3]:
head(data)

,cntry,year_curr,pubcnt_curr,year_prev,pubcnt_prev,growth_rate,China,EU,France,Germany,United.Kingdom,United.States,domestic,others,foreign
,<chr>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,2010,3.496508,2009,2.833213,0.94117647,0.6931472,0.6931472,0.0000000,0.6931472,1.0986123,2.484907,0.6931472,2.197225,3.218876
2,Afghanistan,2011,3.332205,2010,3.496508,-0.15151515,1.9459101,0.6931472,0.0000000,1.9459101,0.6931472,2.397895,0.0000000,1.609438,3.367296
3,Afghanistan,2012,3.663562,2011,3.332205,0.39285714,0.0000000,0.0000000,0.0000000,0.0000000,1.7917595,1.945910,0.0000000,1.098612,2.639057
4,Afghanistan,2013,3.295837,2012,3.663562,-0.30769231,0.0000000,0.0000000,2.1972246,1.3862944,0.6931472,2.302585,0.0000000,2.079442,3.367296
5,Afghanistan,2014,3.761200,2013,3.295837,0.59259259,0.6931472,0.6931472,0.6931472,1.6094379,2.4849066,3.526361,0.0000000,3.465736,4.418841
6,Afghanistan,2015,3.737670,2014,3.761200,-0.02325581,1.7917595,0.0000000,1.7917595,0.0000000,3.4657359,3.433987,0.0000000,4.007333,4.836282


In [4]:
data = subset(data, !(cntry %in% c('China','France','Germany','United States','United Kingdom')))

In [5]:
feols_dome = feols(growth_rate~pubcnt_prev+domestic+foreign|cntry+year_curr,data=data)
summary(feols_dome)

OLS estimation, Dep. Var.: growth_rate
Observations: 1,737 
Fixed-effects: cntry: 201,  year_curr: 9
Standard-errors: Clustered (cntry) 
             Estimate Std. Error   t value  Pr(>|t|)    
pubcnt_prev -1.207478   0.096799 -12.47403 < 2.2e-16 ***
domestic     0.137376   0.030597   4.48983 1.203e-05 ***
foreign      0.532603   0.115451   4.61323 7.071e-06 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.379592     Adj. R2: 0.516959
                 Within R2: 0.468941

In [6]:
feols_fdi = feols(
    growth_rate~pubcnt_prev+domestic+United.States+United.Kingdom+EU+France+Germany+China+others|cntry+year_curr,
    data=data)
summary(feols_fdi)

OLS estimation, Dep. Var.: growth_rate
Observations: 1,737 
Fixed-effects: cntry: 201,  year_curr: 9
Standard-errors: Clustered (cntry) 
                Estimate Std. Error    t value   Pr(>|t|)    
pubcnt_prev    -1.159785   0.095397 -12.157429  < 2.2e-16 ***
domestic        0.133964   0.032044   4.180643 4.3452e-05 ***
United.States   0.180751   0.033778   5.351077 2.3824e-07 ***
United.Kingdom -0.012894   0.024369  -0.529136 5.9730e-01    
EU             -0.030676   0.044908  -0.683101 4.9533e-01    
France          0.018713   0.050896   0.367677 7.1350e-01    
Germany         0.041399   0.033896   1.221347 2.2339e-01    
China           0.032037   0.023747   1.349105 1.7883e-01    
others          0.292930   0.065187   4.493665 1.1834e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.391346     Adj. R2: 0.484552
                 Within R2: 0.435543

In [8]:
notes = "The P-value is derived from the two-tailed t-test. + p < 0.1, * p < 0.05, ** p < 0.01, *** p < 0.001"
models = list(feols_dome, feols_fdi)
modelsummary(models,
             fmt=2,
             estimate  = "{estimate}{stars} [{conf.low},{conf.high}]",
             statistic=NULL,
             coef_rename=c('pubcnt_prev'='Pub(t-1)','United.Kingdom'='UK','United.States'='USA',
             'domestic'='INV_Domestic','foreign'='INV_Foreign','others'='INV_Rest'),
             gof_map=c('nobs','r.squared','adj.r.squared'),
             notes=notes,
             output='kableExtra')%>%kable_styling(latex_options="scale_down") %>%save_kable("regression.pdf")

Note that HTML color may not be displayed on PDF properly.

save_kable will have the best result with magick installed. 



In [38]:
modelsummary(feols_model, fmt=3, statistic='std.error', stars=TRUE,
            
            ,output='kableExtra')%>%kable_styling() %>%save_kable("test.pdf")

Note that HTML color may not be displayed on PDF properly.

save_kable will have the best result with magick installed. 



In [23]:
total = read.csv('fund_grants_total.csv')
total$United.States_fundtotal=log(total$United.States_fundtotal)
total$China_fundtotal=log(total$China_fundtotal)
head(total)

,year,China_fundtotal,EU_fundtotal,France_fundtotal,Germany_fundtotal,United.Kingdom_fundtotal,United.States_fundtotal,others_fundtotal
,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<int>
1,2009,12.75166,27261,33492,63685,44587,12.70250,498743
2,2010,13.00639,39416,45926,81850,61679,12.94872,632475
3,2011,13.27680,48953,54829,94887,72007,13.09072,758037
4,2012,13.54110,52349,57181,99819,78085,13.15509,848046
5,2013,13.78837,59930,58862,101215,80258,13.16273,901959
6,2014,13.97809,64587,58670,101455,78810,13.14795,905909


In [24]:
meta = merge(data, total, by.x='year_curr', by.y='year')

In [27]:
iv_fund=feols(
    growth_rate~pubcnt_prev+China+EU+France+Germany+United.Kingdom+domestic+others|cntry+year_curr|United.States~United.States_fundtotal,
    data=meta)

The instrument 'United.States_fundtotal' have been removed because of collinearity (see $collin.var).



In [28]:
iv_fund$collin.var

[1] "others"                  "United.States_fundtotal"

In [30]:
feols(United.States~United.States_fundtotal+pubcnt_prev+China+EU+France+Germany+United.Kingdom+domestic+others,
      data=meta)

OLS estimation, Dep. Var.: United.States
Observations: 1,737 
Standard-errors: IID 
                         Estimate Std. Error   t value   Pr(>|t|)    
(Intercept)             -0.753091   3.025288 -0.248932 8.0344e-01    
United.States_fundtotal  0.074930   0.230291  0.325372 7.4494e-01    
pubcnt_prev              0.372240   0.024444 15.228153  < 2.2e-16 ***
China                   -0.065904   0.015517 -4.247098 2.2809e-05 ***
EU                      -0.048661   0.026423 -1.841653 6.5697e-02 .  
France                   0.018094   0.018966  0.954000 3.4022e-01    
Germany                  0.033432   0.023975  1.394424 1.6337e-01    
United.Kingdom           0.603346   0.022538 26.769615  < 2.2e-16 ***
domestic                -0.002388   0.018444 -0.129486 8.9699e-01    
others                   0.072818   0.025913  2.810128 5.0080e-03 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.690442   Adj. R2: 0.927037

In [31]:
fund_prev=read.csv('fund_previous.csv')

In [33]:
head(data,2)

,cntry,year_curr,pubcnt_curr,year_prev,pubcnt_prev,growth_rate,China,EU,France,Germany,United.Kingdom,United.States,domestic,others
,<chr>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,2010,3.496508,2009,2.833213,0.9411765,0.6931472,0.6931472,0,0.6931472,1.0986123,2.484907,0.6931472,2.197225
2,Afghanistan,2011,3.332205,2010,3.496508,-0.1515152,1.9459101,0.6931472,0,1.9459101,0.6931472,2.397895,0.0000000,1.609438


In [34]:
head(fund_prev,2)

,author_distinct,year,China_prev,EU_prev,France_prev,Germany_prev,United.Kingdom_prev,United.States_prev,domestic_prev,others_prev
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,2009,0.0000000,0.0000000,0,0.6931472,1.945910,1.098612,0.0000000,0.6931472
2,Afghanistan,2010,0.6931472,0.6931472,0,0.6931472,1.098612,2.484907,0.6931472,2.1972246


In [35]:
iv_df2=merge(data,fund_prev, by.x=c("cntry","year_prev"),by.y=c("author_distinct",'year'))

In [55]:
feols(domestic~domestic_prev+pubcnt_prev+China+EU+France+Germany+United.Kingdom+United.States+others,
      data=iv_df2)

OLS estimation, Dep. Var.: domestic
Observations: 1,719 
Standard-errors: IID 
                Estimate Std. Error   t value   Pr(>|t|)    
(Intercept)    -0.221166   0.048900 -4.522784 6.5233e-06 ***
domestic_prev   0.887502   0.011552 76.826056  < 2.2e-16 ***
pubcnt_prev     0.066948   0.016951  3.949508 8.1494e-05 ***
China           0.006312   0.009519  0.663028 5.0740e-01    
EU             -0.032021   0.016641 -1.924264 5.4487e-02 .  
France          0.015784   0.011804  1.337109 1.8137e-01    
Germany         0.023731   0.014895  1.593194 1.1130e-01    
United.Kingdom  0.019514   0.016753  1.164801 2.4426e-01    
United.States  -0.011493   0.015128 -0.759696 4.4754e-01    
others          0.054893   0.016372  3.352771 8.1754e-04 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.429396   Adj. R2: 0.984108

In [67]:
iv_prev=feols(
    growth_rate~pubcnt_prev+United.States+EU+France+Germany+United.Kingdom+others+China|cntry+year_curr|domestic~domestic_prev,
    data=iv_df2,cluster=c('cntry','year_curr'))

In [68]:
summary(iv_prev)

TSLS estimation, Dep. Var.: growth_rate, Endo.: domestic, Instr.: domestic_prev
Second stage: Dep. Var.: growth_rate
Observations: 1,719 
Fixed-effects: cntry: 201,  year_curr: 9
Standard-errors: Clustered (cntry & year_curr) 
                Estimate Std. Error   t value   Pr(>|t|)    
fit_domestic    0.364811   0.239945  1.520393 1.6690e-01    
pubcnt_prev    -1.171933   0.135083 -8.675641 2.4252e-05 ***
United.States   0.138809   0.037294  3.721991 5.8546e-03 ** 
EU             -0.033706   0.045781 -0.736237 4.8261e-01    
France          0.012840   0.056046  0.229096 8.2454e-01    
Germany         0.044886   0.037017  1.212577 2.5988e-01    
United.Kingdom -0.016125   0.032824 -0.491252 6.3644e-01    
others          0.261515   0.089285  2.929005 1.9027e-02 *  
China           0.025206   0.020847  1.209108 2.6114e-01    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.390417     Adj. R2: 0.426566
                 Within R2: 0.376703
F-test (1st stage), do

In [69]:
summary(iv_prev, stage=1)

TSLS estimation, Dep. Var.: domestic, Endo.: domestic, Instr.: domestic_prev
First stage: Dep. Var.: domestic
Observations: 1,719 
Fixed-effects: cntry: 201,  year_curr: 9
Standard-errors: Clustered (cntry & year_curr) 
                Estimate Std. Error   t value  Pr(>|t|)    
domestic_prev   0.258896   0.065981  3.923808 0.0043948 ** 
pubcnt_prev     0.123614   0.034854  3.546656 0.0075462 ** 
United.States   0.040910   0.034987  1.169289 0.2759444    
EU             -0.000235   0.029870 -0.007876 0.9939088    
France          0.020149   0.023127  0.871250 0.4090011    
Germany         0.007584   0.021347  0.355289 0.7315584    
United.Kingdom  0.038567   0.025720  1.499469 0.1721379    
others          0.127580   0.027999  4.556554 0.0018583 ** 
China           0.021209   0.007177  2.954994 0.0182856 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.330739     Adj. R2: 0.989266
                 Within R2: 0.202147
F-test (1st stage): stat = 129.5, p < 2

In [65]:
feols_model = feols(
    growth_rate~pubcnt_prev+China+EU+France+Germany+United.Kingdom+United.States+domestic+others+others_prev|cntry+year_curr,
    data=iv_df2)

In [66]:
summary(feols_model)

OLS estimation, Dep. Var.: growth_rate
Observations: 1,719 
Fixed-effects: cntry: 201,  year_curr: 9
Standard-errors: Clustered (cntry) 
                Estimate Std. Error   t value   Pr(>|t|)    
pubcnt_prev    -1.078086   0.113629 -9.487802  < 2.2e-16 ***
China           0.031676   0.023220  1.364178 1.7404e-01    
EU             -0.018040   0.044368 -0.406597 6.8474e-01    
France          0.027881   0.048773  0.571643 5.6821e-01    
Germany         0.050855   0.033439  1.520834 1.2988e-01    
United.Kingdom -0.007303   0.022578 -0.323480 7.4667e-01    
United.States   0.148512   0.030897  4.806612 3.0109e-06 ***
domestic        0.139200   0.032243  4.317172 2.4852e-05 ***
others          0.298562   0.065434  4.562794 8.7974e-06 ***
others_prev    -0.066167   0.029751 -2.224019 2.7266e-02 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.381718     Adj. R2: 0.451468
                 Within R2: 0.404168